In [1]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
# from nltk.tokenize import TweetTokenizer
from nltk.tokenize.casual import reduce_lengthening
# from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from math import ceil
from sklearn.metrics import (f1_score, classification_report, 
                            confusion_matrix, accuracy_score, 
                            precision_score, recall_score, 
                            roc_auc_score, roc_curve)
from sklearn.preprocessing import label_binarize
from scipy.spatial.distance import cosine
import string
from math import log10,sqrt
from scipy.stats import norm
import re
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
columns = ['tweet_id','timestamp','tweet_text','user_id',
           'tweet_coords','tweet_coords_list','tweet_long','tweet_lat','location',
           'enc_url','tweet_lang','hashtags']
tweet_full = pd.read_csv(r'./tweetCoords.csv',
                         header=None,
                         names=columns,
                         parse_dates=[1],
                         infer_datetime_format=True,
                         index_col='timestamp')

In [3]:
related_words = pd.DataFrame()
tweet_date = pd.to_datetime("2017-09-10 00:00:00")
date_delta = pd.Timedelta("24HR")
end_date = pd.to_datetime("2017-09-10 00:00:00")

In [4]:
tweet_full_en = tweet_full[tweet_full['tweet_lang'] == 'en']

In [5]:
tweet_text = tweet_full_en.loc[tweet_date:tweet_date + date_delta,"tweet_text"]

In [6]:
tweet_stops = stopwords.words('english')
stemmer = PorterStemmer()

def clean_tweet(tweet):
##     takes input string and converts or removes characters depending on settings.
##     returns a string
##     convert case:
    tweet = tweet.lower()
##    remove URLs:
    tweet = re.sub('https?://\S+','',tweet)
##     remove @mentions, including those with a leading '-' or '.' : 
    tweet = re.sub('[-\.]?@\w+','',tweet)
##     remove non-hashtag punctuation:
#     tweet = tweet.translate(tweet.maketrans('','',string.punctuation.replace("#","")))
##     convert non-hashtag punctuation to whitespace:
    tweet = tweet.translate(tweet.maketrans(string.punctuation.replace("#","")," "*len(string.punctuation.replace("#",""))))
#     remove non-hashtag '#'.
    tweet = re.sub('\B#\B','',tweet)
##     remove 'amp', 'gt', 'lt', indicating decoded ampersand, greater-than, less-than characters
    tweet = re.sub(r'\b(amp|gt|lt)\b','',tweet)
##     drop numbers and words of < 4 characters.
#     tweet = re.sub(r'\b(?<!#)\w{1,3}\b','',tweet)
    tweet = re.sub(r'\b(?<!#)\d+\b','',tweet)
    return tweet

def tokens_no_stopwords(tweet_as_string):
#     wrapper function that combines the tokenizer, cleaner, and stopword removal.
#     takes a string and returns a list of strings
    cleaned_tweet = clean_tweet(tweet_as_string)
    tweet_reduce_len = reduce_lengthening(cleaned_tweet)
#     tweet_as_tokens = word_tokenize(tweet_reduce_len)
    tweet_as_tokens = tweet_reduce_len.split()
    tweet_no_stops = [stemmer.stem(word) for word in tweet_as_tokens if word not in tweet_stops]
    
    return tweet_no_stops

In [7]:
search_term = 'irma'

In [8]:
tweet_text = tweet_full_en.loc[tweet_date:tweet_date + date_delta,"tweet_text"]
tweets_tokens = tweet_text.apply(tokens_no_stopwords)

ns_size = 1
# ---------- Skip-Gram ----------
# initialize the Word2Vec vectors with window size 7, min count 1, and Skip-Gram (Negative Sampling Test)
vector_model = Word2Vec(tweets_tokens, min_count=1, window=7, workers=1, size=100, seed=1, sg=1, negative=ns_size)

# train the model over 10 epochs
vector_model.train(tweets_tokens, total_examples=len(tweet_text), epochs=10)

# assign the vector data to its position in a series
# vectors_series.iloc[i] = vector_model

(1180791, 1289640)

In [9]:
# tweet_text = tweet_full_en.loc[tweet_date:tweet_date + date_delta,"tweet_text"]
# tweets_tokens = tweet_text.apply(tokens_no_stopwords)
# # ---------- Skip-Gram ----------
# # initialize the Word2Vec vectors with window size i, min count 1, and Skip-Gram
# vector_model = Word2Vec(tweets_tokens, min_count=1, window=5, workers=1, size=100, seed=1, negative=1)

# # train the model over 10 epochs
# vector_model.train(tweets_tokens, total_examples=len(tweet_text), epochs=10)

In [10]:
vector_model.wv.most_similar(search_term)

/Users/brown/.local/share/virtualenvs/TwitterDisaster-4Cppn-LV/lib/python3.7/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('pass', 0.7625612020492554),
 ('hit', 0.7501078844070435),
 ('move', 0.7244436740875244),
 ('eye', 0.7225500345230103),
 ('away', 0.7124370336532593),
 ('shortli', 0.7074049115180969),
 ('around', 0.6974090337753296),
 ('slightli', 0.6877397298812866),
 ('wall', 0.6838585138320923),
 ('direct', 0.6806619167327881)]

In [11]:
coded_tweets = pd.read_csv(r'./irma_related_tweets.csv')

tweets_on_date = tweet_full_en.loc[tweet_date:tweet_date+date_delta]

tweet_encoded = pd.concat([coded_tweets.reset_index(),tweets_on_date.iloc[:-1].reset_index()],axis=1)

In [12]:
# func_type = "Mean Cosine Similarity"
# func_short = "MeanCosSim"
def ScoreTweetFromVectorsMSC(tweet,vector_set):
    tweet_as_terms = tokens_no_stopwords(tweet)
    score = 0
    for i in tweet_as_terms:
        if i in vector_set.wv.vocab:
            score += vector_set.wv.similarity(i,search_term)
    if len(tweet_as_terms) > 0:
        score /= len(tweet_as_terms)
    else:
        score = 0
    return score

In [13]:
# func_type = "Dot Product of Tweet Vector and Search Term"
# func_short = "DotProduct"
def ScoreTweetFromVectorsDP(tweet,vector_set):
    tweet_as_terms = tokens_no_stopwords(tweet)
#     initialize vector with dimensionality of the vector set.
    vector_dim = len(vector_set.wv.vectors[0])
    score_matrix = np.zeros(vector_dim,) 
#     iterate over each word after processing. If the word is in the vocabulary,
#     add its vector's value to the score matrix.
#     this essentially treats a word not in the vocabulary as a zero-vector.
    for i in tweet_as_terms:
        if i in vector_set.wv.vocab:
            score_matrix = np.add(score_matrix,vector_set.wv.get_vector(i))
#     if the number of words remaining in the tweet after processing is equal to zero, return zero.
#     otherwise, take the dot product of the score vector, and the vector of the search term.
    if len(tweet_as_terms) > 0:
        score = np.dot(score_matrix,vector_set.wv.get_vector(search_term))
    else:
        score = 0
    return score

In [14]:
# func_type = "Cosine Similarity of Tweet Vector Sum and Search Term"
# func_short = "CosSimTweetVecSum"
def ScoreTweetFromVectorsCSTVS(tweet,vector_set):
    tweet_as_terms = tokens_no_stopwords(tweet)
#     initialize vector with dimensionality of the vector set.
    vector_dim = len(vector_set.wv.vectors[0])
    score_matrix = np.zeros(vector_dim,) 
#     iterate over each word after processing. If the word is in the vocabulary,
#     add its vector's value to the score matrix.
#     this essentially treats a word not in the vocabulary as a zero-vector.
    for i in tweet_as_terms:
        if i in vector_set.wv.vocab:
            score_matrix = np.add(score_matrix,vector_set.wv.get_vector(i))
#     if the number of words remaining in the tweet after processing is equal to zero, return zero.
#     otherwise, take the pairwise cosine of the score vector and the vector of the search term.
    if len(tweet_as_terms) > 0:
        score = 1 - cosine(score_matrix,vector_set.wv.get_vector(search_term))
    else:
        score = 0
    return score

In [15]:
# func_type = "Sum Score over Sqrt Cos. Sim."
# func_short = "SumOverSqrtLen"
def ScoreTweetFromVectorsSSSCS(tweet,vector_set):
    tweet_as_terms = tokens_no_stopwords(tweet)
    score = 0
    for i in tweet_as_terms:
        if i in vector_set.wv.vocab:
            score += vector_set.wv.similarity(i,search_term)
    if len(tweet_as_terms) > 0:
        score /= sqrt(len(tweet_as_terms))
    else:
        score = 0
    return score

In [16]:
#      Scoring tweets in this column:
score_type = 'MSC'
sw = f'score_window_{score_type}'
mmsw = f'MM_{sw}'
tweet_encoded[sw] = tweet_encoded.tweet_text.apply(ScoreTweetFromVectorsMSC,args=(vector_model,))

#     column of scores for this iteration
tweet_scores = tweet_encoded[sw]

#     calculating Min Max Scaling for this column 
tweet_encoded[mmsw] = ((tweet_scores - tweet_scores.min())* 100) / (tweet_scores.max() - tweet_scores.min())

score_type = 'DP'
sw = f'score_window_{score_type}'
mmsw = f'MM_{sw}'
tweet_encoded[sw] = tweet_encoded.tweet_text.apply(ScoreTweetFromVectorsDP,args=(vector_model,))

#     column of scores for this iteration
tweet_scores = tweet_encoded[sw]

#     calculating Min Max Scaling for this column 
tweet_encoded[mmsw] = ((tweet_scores - tweet_scores.min())* 100) / (tweet_scores.max() - tweet_scores.min())

score_type = 'CSTVS'
sw = f'score_window_{score_type}'
mmsw = f'MM_{sw}'
tweet_encoded[sw] = tweet_encoded.tweet_text.apply(ScoreTweetFromVectorsCSTVS,args=(vector_model,))

#     column of scores for this iteration
tweet_scores = tweet_encoded[sw]

#     calculating Min Max Scaling for this column 
tweet_encoded[mmsw] = ((tweet_scores - tweet_scores.min())* 100) / (tweet_scores.max() - tweet_scores.min())

score_type = 'SSSCS'
sw = f'score_window_{score_type}'
mmsw = f'MM_{sw}'
tweet_encoded[sw] = tweet_encoded.tweet_text.apply(ScoreTweetFromVectorsSSSCS,args=(vector_model,))

#     column of scores for this iteration
tweet_scores = tweet_encoded[sw]

#     calculating Min Max Scaling for this column 
tweet_encoded[mmsw] = ((tweet_scores - tweet_scores.min())* 100) / (tweet_scores.max() - tweet_scores.min())

In [17]:
tweet_encoded

index  irma_rel                                              tweet  \
0          0         1  Wind 2.0 mph N. Barometer 29.880 in, Steady. T...   
1          1         0  There is always beauty regardless the circumst...   
2          2         0  #Carpool #Orlando - gt  #Orlando #KIrkmanRoad ...   
3          3         0  @avictoria_x nah i m chillin my eyes never get...   
4          4         0    @jvnvy @sza That s how I feel bout seein travis   
5          5         1  Taking a beak from boarding the windows. Kitty...   
6          6         1  19:53 Temp. 82.4 F, Hum. 79 , Dewp. 73.6 F, Ba...   
7          7         1  I m so proud of our local radio, news, animal ...   
8          8         1  Evening of a Hurricane. #hurricane #hurricanei...   
9          9         0           hey big head     https://t.co/M46uNJoxx2   
10        10         1  Deplorable: deplorable nation : please pray fo...   
11        11         0    All I ve ever wanted    https://t.co/ggjica11Lz   
12        12         0                              Lol what is this game   
13        13         1  I m about to put on a swimsuit and go outside....   
14        14         0  It looks like you do just fine in Kissimmee. B...   
15        15         0          THE FFUCK IS THIS https://t.co/LVhxRyLxvc   
16        16         0  It has to be 3-6 players   https://t.co/VzTEQw...   
17        17         1                Ayo #HurrcaneIrma where you at tho    
18        18         0  Just posted a photo @ Miami South Beach, Flori...   
19        19         1  temperature up 82 F - gt  84 F humidity down 8...   
20        20         1  These news casters don t get paid enough for this   
21        21         0  DONATE THIS CHANNEL  https://t.co/hchxq37CpI v...   
22        22         0                              @jvnvy Yeeeeeeehawwww   
23        23         0       CARLOOOOSSSS LMFAOOO https://t.co/TM7gr5XEN9   
24        24         1  Can t even watch college football cause all th...   
25        25         0  This will be a much different view tomorrow ni...   
26        26         1  #tampa #hurricaneirma #sunset #bestwishes #sta...   
27        27         1  If I see that hurricane look on my TL again I ...   
28        28         0  DONATE THIS CHANNEL  https://t.co/hchxq37CpI v...   
29        29         0  Styx still open done got me a drink and finna ...   
...      ...       ...                                                ...   
19058  19058         1             Irma does hair https://t.co/QQssT2BuPR   
19059  19059         1  #HurricaneIrma starting to make waves in East ...   
19060  19060         0  Donate this channel https://t.co/GtKdocp2vl @ ...   
19061  19061         1  How do you fully prepare for a beast like #Hur...   
19062  19062         0                             I m accidentally drunk   
19063  19063         0      i can t even get in contact wit my peoples      
19064  19064         1  @insideFPL Thank you for getting my power back...   
19065  19065         0  Donate this channel https://t.co/GtKdoc7rDN @ ...   
19066  19066         1  7:30 update power has gone out once but came b...   
19067  19067         1                            Power is officially out   
19068  19068         1  @cityofaventura New curfew in Miami-Dade Miami...   
19069  19069         0  Donate this channel https://t.co/GtKdoc7rDN @ ...   
19070  19070         1                   Electricity out bout to get real   
19071  19071         1  Winds definitely picking up. #HurricaneIrma @ ...   
19072  19072         0  Fat bottomed mugs, you make the caffeine world...   
19073  19073         1  Going to have alcohol poisoning before this hu...   
19074  19074         0  There are still cars on the road. Where are yo...   
19075  19075         1  Cape Canaveral,FL  XMR  ASOS reports gust of 5...   
19076  19076         1  Cape Canaveral,FL  XMR  ASOS reports gust of 5...   
19077  19077         1  Irmagerd it s my pack     @ Saint Augustine Sh...   
190

In [18]:
test_cols = ['tweet','irma_rel','MM_score_window_MSC','MM_score_window_DP','MM_score_window_CSTVS','MM_score_window_SSSCS']

In [19]:
tweet_encoded[test_cols]

tweet  irma_rel  \
0      Wind 2.0 mph N. Barometer 29.880 in, Steady. T...         1   
1      There is always beauty regardless the circumst...         0   
2      #Carpool #Orlando - gt  #Orlando #KIrkmanRoad ...         0   
3      @avictoria_x nah i m chillin my eyes never get...         0   
4        @jvnvy @sza That s how I feel bout seein travis         0   
5      Taking a beak from boarding the windows. Kitty...         1   
6      19:53 Temp. 82.4 F, Hum. 79 , Dewp. 73.6 F, Ba...         1   
7      I m so proud of our local radio, news, animal ...         1   
8      Evening of a Hurricane. #hurricane #hurricanei...         1   
9               hey big head     https://t.co/M46uNJoxx2         0   
10     Deplorable: deplorable nation : please pray fo...         1   
11       All I ve ever wanted    https://t.co/ggjica11Lz         0   
12                                 Lol what is this game         0   
13     I m about to put on a swimsuit and go outside....         1   
14     It looks like you do just fine in Kissimmee. B...         0   
15             THE FFUCK IS THIS https://t.co/LVhxRyLxvc         0   
16     It has to be 3-6 players   https://t.co/VzTEQw...         0   
17                   Ayo #HurrcaneIrma where you at tho          1   
18     Just posted a photo @ Miami South Beach, Flori...         0   
19     temperature up 82 F - gt  84 F humidity down 8...         1   
20     These news casters don t get paid enough for this         1   
21     DONATE THIS CHANNEL  https://t.co/hchxq37CpI v...         0   
22                                 @jvnvy Yeeeeeeehawwww         0   
23          CARLOOOOSSSS LMFAOOO https://t.co/TM7gr5XEN9         0   
24     Can t even watch college football cause all th...         1   
25     This will be a much different view tomorrow ni...         0   
26     #tampa #hurricaneirma #sunset #bestwishes #sta...         1   
27     If I see that hurricane look on my TL again I ...         1   
28     DONATE THIS CHANNEL  https://t.co/hchxq37CpI v...         0   
29     Styx still open done got me a drink and finna ...         0   
...                                                  ...       ...   
19058             Irma does hair https://t.co/QQssT2BuPR         1   
19059  #HurricaneIrma starting to make waves in East ...         1   
19060  Donate this channel https://t.co/GtKdocp2vl @ ...         0   
19061  How do you fully prepare for a beast like #Hur...         1   
19062                             I m accidentally drunk         0   
19063      i can t even get in contact wit my peoples            0   
19064  @insideFPL Thank you for getting my power back...         1   
19065  Donate this channel https://t.co/GtKdoc7rDN @ ...         0   
19066  7:30 update power has gone out once but came b...         1   
19067                            Power is officially out         1   
19068  @cityofaventura New curfew in Miami-Dade Miami...         1   
19069  Donate this channel https://t.co/GtKdoc7rDN @ ...         0   
19070                   Electricity out bout to get real         1   
19071  Winds definitely picking up. #HurricaneIrma @ ...         1   
19072  Fat bottomed mugs, you make the caffeine world...         0   
19073  Going to have alcohol poisoning before this hu...         1   
19074  There are still cars on the road. Where are yo...         0   
19075  Cape Canaveral,FL  XMR  ASOS reports gust of 5...         1   
19076  Cape Canaveral,FL  XMR  ASOS reports gust of 5...         1   
19077  Irmagerd it s my pack     @ Saint Augustine Sh...         1   
19078  #armystrong #passtheguinness @ Windermere, Flo...         1   
19079     en Miami Beach, Florida https://t.co/B3LN83...         0   
19080  I finished the Tonga - Premier championship  @...         0   
19081  This will definitely be a #Birthday to remembe...         0   
19082  @AmyEarnhardt @Buckeyefanohio7 Key West is ok,...         1   
19083                            SPRINT FUCKING SUCKS :      

In [20]:
for i in ['MSC','DP','CSTVS','SSSCS']:
    print(roc_auc_score(tweet_encoded['irma_rel'],tweet_encoded[f'MM_score_window_{i}']))

0.6904583907290875
0.8464484455966883
0.7467259177304093
0.7936167782592627


In [21]:
fig_roc_1 = plt.figure(figsize=(12,12),facecolor='w')
for i in ['MSC','DP','CSTVS','SSSCS']:
    fpr, tpr, thresholds = roc_curve(tweet_encoded['irma_rel'],tweet_encoded[f'MM_score_window_{i}'])
    plt.plot(fpr,tpr,label=f"{i}")
plt.plot([0,1],[0,1],linestyle="-.",label=".5 Threshold")
plt.legend()
plt.show()
# fig_roc_1.savefig(f'./figures/poster_roc_curve.pdf',bbox_inches='tight')

In [22]:
csv_cols = ['tweet_id','irma_rel','MM_score_window_DP']

In [26]:
tweet_encoded[csv_cols].to_csv('./poster_text.csv')

In [25]:
%pwd

'/Users/brown/Documents/TwitterDisaster'